# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

Before running any of the following code, make sure you import the dataset with 
`mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`
<!-- `mongoimport --type json -d met -c artifacts --drop --jsonArray artifacts.json` -->

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'trabel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65835793c397fa253815ac4d'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new)

In [17]:
# Check that the new restaurant was inserted
results  = establishments.find(new)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6583597a21f3b261257b0fc9'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [21]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# query = {'BusinessTypeID': {'$regex': "Restaurant/Cafe/Canteen"}}
query = {'BusinessType': {'$regex': "Restaurant/Cafe/Canteen"}}
fields = {"BusinessTypeID": 1, "BusinessType": 1}
sorts = sort = [("BusinessTypeID", -1)]

#Results shows that the new restaurant has an empty BusinessTypeID field
results = list(establishments.find(query, fields).sort(sorts).limit(5))
pprint(results)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  '_id': ObjectId('6583597a21f3b261257b0fc9')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65835793c397fa253815ac58')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65835793c397fa253815ac59')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65835793c397fa253815ac55')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65835793c397fa253815ac4f')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [22]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(new, {'$set': {'BusinessTypeID': 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [26]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName': 'Penang Flavours'})


{'_id': ObjectId('6583597a21f3b261257b0fc9'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"


In [ ]:
# Check if any remaining documents include Dover


In [ ]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers
